# Moldel Training

In [5]:
import pandas as pd
import pycaret.classification as pc

In [6]:
df = pd.read_csv('../../src/output/alunos_final.csv')
df.head(1)

,status,ccr,nome_docente,freq_turma,turno
0,1,Algebra linear - GEX607,ANTONIO MARCOS CORREA NERI,91.67,NOTURNO


In [7]:
s = pc.setup(df, target='status', 
          iterative_imputation_iters=10, 
          session_id=123,  
          train_size=0.99, 
          remove_outliers=True, 
          outliers_method='iforest',
          categorical_features=['ccr', 'nome_docente', 'turno'],
          numeric_features=['freq_turma'],
          fold_strategy='stratifiedkfold',
          fold=10,
          fold_shuffle=True,
          n_jobs=4,
          ) # use_gpu=True,

,Description,Value
0,Session id,123
1,Target,status
2,Target type,Binary
3,Original data shape,"(18148, 5)"
4,Transformed data shape,"(17249, 5)"
5,Transformed train set shape,"(17067, 5)"
6,Transformed test set shape,"(182, 5)"
7,Ordinal features,1
8,Numeric features,1
9,Categorical features,3


In [8]:
lightgbm = pc.create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8575,0.9390,0.7385,0.8741,0.8006,0.6910,0.6970
1,0.8587,0.9415,0.7543,0.8635,0.8052,0.6951,0.6990
2,0.8687,0.9408,0.7759,0.8710,0.8207,0.7176,0.7206
3,0.8531,0.9350,0.7446,0.8579,0.7972,0.6830,0.6872
4,0.8631,0.9399,0.7618,0.8691,0.8119,0.7050,0.7088
5,0.8564,0.9390,0.7403,0.8702,0.8000,0.6892,0.6946
6,0.8497,0.9329,0.7443,0.8492,0.7933,0.6760,0.6796
7,0.8703,0.9465,0.7902,0.8634,0.8252,0.7224,0.7242
8,0.8630,0.9358,0.7500,0.8788,0.8093,0.7035,0.7089


In [9]:
pc.evaluate_model(lightgbm)
lightgbm

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=4, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [10]:
pc.save_model(lightgbm, '../../src/models/LGBMClassifier')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None, include=['freq_turma'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean',
                                                               verbose='deprecated'))),
                 ('categorical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['ccr', 'n...
                  LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                 colsample

In [17]:
pc.plot_model(lightgbm, plot='feature', scale=10, save=True)
pc.plot_model(lightgbm, plot='confusion_matrix', plot_kwargs = {'percent' : True}, save=True, scale=10)


values = pc.predict_model(lightgbm)
test = pc.pull()

#test is a pandas dataframe, print it to html

test.to_html('../../src/output/values.html')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8956,0.9545,0.8592,0.8714,0.8652,0.7801,0.7801


In [12]:
# print values to csv

values = values.sort_values(by=['ccr', 'nome_docente'], ascending=True) #, 'ano'

values.to_csv('../../src/output/values.csv', index=False)

# Predição

Caso já tenha executado os passos anteriores, basta executar as celulas abaixo para realizar a predição.

In [2]:
import ipywidgets as widgets
from IPython.display import HTML
import pandas as pd
from pycaret.classification import predict_model, load_model

In [22]:
df = pd.read_csv('../../src/output/alunos_final.csv')
ccr = widgets.Dropdown(options = df['ccr'].sort_values().unique().tolist(), description='CCR:')
nome_docente = widgets.Dropdown(options = df['nome_docente'].sort_values().unique().tolist(), description='Docente:')
frequencia = widgets.SelectionSlider(options = df['freq_turma'].sort_values().unique().tolist(), description='Frequência:', disabled=False, continuous_update=True, orientation='horizontal', readout=True)
turno = widgets.Dropdown(options = df['turno'].sort_values().unique().tolist(), description='Turno:')
widgets.HBox([ccr, nome_docente, turno, frequencia])

In [34]:
# create a json file with the values of the dropdown menus
input_data = {
    'ccr': ccr.value,
    'nome_docente': nome_docente.value,
    'freq_turma': frequencia.value,
    'turno': turno.value,
}

prediction_model = load_model('../../src/models/LGBMClassifier')

input_data = pd.DataFrame([input_data])
    
new_prediction = predict_model(prediction_model, data=input_data)

# rename the column 'prediction_label' to 'status' and put in index 0 using insert and pop method
new_prediction.insert(0, 'status', new_prediction.pop('prediction_label'))

Transformation Pipeline and Model Successfully Loaded


In [35]:
HTML(new_prediction.to_html(index=False))

status,ccr,nome_docente,freq_turma,turno,prediction_score
0,Probabilidade e estatística - GEX055,LEANDRO BORDIN,73.330002,VESPERTINO,0.9166
